In [1]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
caminhos =[
    "../arquivos/Explorando o Universo das IAs com Hugging Face.pdf",
    "../arquivos/Apostila LangChain.pdf",
    "../arquivos/Explorando a API da OpenAI.pdf",
    "../arquivos/Explorando a API da OpenAI.pdf"
]

paginas = []

for caminho in caminhos:
    loader = PyPDFLoader(caminho)
    paginas.extend(loader.load())
    
recur_split= RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", " ", ""]
)

documents = recur_split.split_documents(paginas)

Enumerar todos os arquivos do diretório atual

In [4]:
for i, doc in enumerate(documents):
    doc.metadata["source"] = doc.metadata["source"].replace("../arquivos/", "")
    doc.metadata["doc_id"] = i

In [ ]:
documents[0].metadata

In [6]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

embedding_model = OpenAIEmbeddings()

In [7]:
diretorio = '../arquivos/chroma_retrieval_bd'

vectordb = Chroma.from_documents(
    documents = documents,
    embedding=embedding_model,
    persist_directory=diretorio
)

In [ ]:
pergunta = "O que é o OpenAI?"

docs = vectordb.similarity_search(pergunta, k= 3)

for doc in docs:
    print(doc.page_content)
    print(f"======================{doc.metadata}", "\n")

### MMR

In [ ]:
pergunta = "O que é o OpenAI?"

docs = vectordb.max_marginal_relevance_search(pergunta, k=3, fetch_k=10)

for doc in docs:
    print(doc.page_content)
    print(f"======================{doc.metadata}\n \n")

### Filtragem de dados

In [ ]:
pergunta = "O que a Apostila de Hugging Face fala sobre a OpenAI eo ChatGPT?"

docs = vectordb.similarity_search(
    pergunta,
    k=3,
    filter= {"source": "Explorando o Universo das IAs com Hugging Face.pdf"}
    )

for doc in docs:
    print(doc.page_content)
    print(f"======================{doc.metadata}", "\n")
    print("\n")

### LLM Aided Retrieval


In [26]:
from langchain_openai.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.schema import AttributeInfo

In [31]:
metadata_info =[
    AttributeInfo(
        name="source",
        description="Nome da apostila de onde o texto original foi retirado. O valor de : Explorando o Universo das IAs com Hugging Face.pdf ou Explorando a API da OpenAI.pdf",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="A página onde o texto original foi retirado. O valor de : 1 ou 2",
        type="integer",
    ),
]

In [32]:
document_description = "Apostila de Cursos"

llm = OpenAI()
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_description,
    metadata_info,
    verbose=True
)

In [ ]:
pergunta = "O que a Apostila de Hugging Face fala sobre a OpenAI eo ChatGPT?"

docs = retriever.get_relevant_documents(pergunta)

for doc in docs:
    print(doc.page_content)
    print(f"======================{doc.metadata}", "\n")
    print("\n")